# Libs. to import

In [6]:
import time
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA,SLSQP
from qiskit_ibm_runtime import EstimatorV2
from qiskit.primitives import Estimator
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import Aer
from qiskit_aer import AerSimulator, QasmSimulator, StatevectorSimulator
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, runtime_job
from qiskit_nature.second_q.hamiltonians import HeisenbergModel
from qiskit_nature.second_q.problems import LatticeModelProblem
from qiskit_nature.second_q.mappers import *
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit import visualization
from qiskit_algorithms import NumPyMinimumEigensolver
import numpy as np
from math import pi
import rustworkx as rx
from qiskit_nature.second_q.hamiltonians.lattices import ( BoundaryCondition, 
Lattice,
LatticeDrawStyle,
LineLattice)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
import pandas as pd
import ace_tools_open as tools

# IBM Account Loading

In [7]:
QiskitRuntimeService.save_account(token="AIP",channel = 'ibm_quantum' ,overwrite=True)
service = QiskitRuntimeService()

# Fininding availabe quantum processors according to the number of spins

In [8]:
# Set the number of spins (nodes)
num_nodes =2 # Start with 8 qubits for real quantum device

# List available backends
backends = service.backends()
for backend in backends:
    print(backend.name)

# Select a real quantum backend with at least num_nodes qubits
backend = service.least_busy(min_num_qubits=num_nodes)
print(f"Using backend: {backend.name}")

ibm_brisbane
ibm_kyiv
ibm_sherbrooke
Using backend: ibm_brisbane


In [11]:
#if need to run the code on simulator not real quantum computer uncomment the following step
#backend = AerSimulator(method='matrix_product_state')
backend = StatevectorSimulator()
#backend = QasmSimulator()
backend

StatevectorSimulator('statevector_simulator')

# Preparing the Systsm

In [47]:
no_Spins = 20

# Define The lattice 
line_latticec = LineLattice(num_nodes=no_Spins, boundary_condition=BoundaryCondition.PERIODIC)

# Construct the Heisenberg Hamiltonian

interaction_coefficients = (1,1,1) #J on x,y and z axis 
external_magntic_field = (0,0,0) # magnetic field

heisenberg_model = HeisenbergModel (line_latticec, interaction_coefficients, external_magntic_field)

#Generate spin Hamiltonian to map it on the qubits 
spin_ham = heisenberg_model.second_q_op()


print (f'the system is defined, number of spins:  {no_Spins} \n Hiesenberg Hamiltonian: {heisenberg_model.interaction_matrix()} \n Spin Hamiltonian: {spin_ham}')

the system is defined, number of spins:  20 
 Hiesenberg Hamiltonian: [[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.

# Mapping the system into quantum circuit 

In [48]:
mapper = LogarithmicMapper()
qubit_op = mapper.map(spin_ham)

print (f'The number of qubits in the qubit operator is {qubit_op.num_qubits}')

The number of qubits in the qubit operator is 20


# Finding the Ground State Energy Using Viriational Quantum Eigensolver (VQE)
VQE requires three inputs to find the miniumum eigenvalue (ground state energy): \
1- Estimator\
2- Ansatz\
3- Optimizer 

In [49]:
# These steps to prepare the system for VQE 
estimator = Estimator(options={"backend":backend, "shots":1024})
ansatz = EfficientSU2(qubit_op.num_qubits, reps=2)
#print(f'number of qubits in ansatz is: {ansatz}')
#This step to make the quantum circuit most efficient for the quantum process we chose previously
transpiled_ansatz = transpile (ansatz,backend=backend,optimization_level=3)
optimizer = SLSQP(maxiter=200) 

print(f"Number of qubits in the qubit operator: {qubit_op.num_qubits}")
print(f"Number of qubits in the ansatz: {ansatz.num_qubits}") 


C:\Users\msi\AppData\Local\Temp\ipykernel_21616\3595077429.py:2: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"backend":backend, "shots":1024})


Number of qubits in the qubit operator: 20
Number of qubits in the ansatz: 20


In [ ]:
vqe = VQE(estimator, ansatz , optimizer)

start_time = time.time()
#Compute the minimum eigenvalue 
result = vqe.compute_minimum_eigenvalue(qubit_op)
end_time=time.time()

time_taken = end_time - start_time 

total_energy= result.eigenvalue.real
energy_per_spin = total_energy/no_Spins
print (total_energy)
print (energy_per_spin)
print (f"energy calcaulation time is: {time_taken} seconds")

# These are some tests 

## This test contains the algorithm in one cell

In [18]:
import time
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, SLSQP
from qiskit.primitives import Estimator
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_nature.second_q.hamiltonians import HeisenbergModel
from qiskit_nature.second_q.mappers import LogarithmicMapper
from qiskit_nature.second_q.hamiltonians.lattices import BoundaryCondition, LineLattice

# Use AerSimulator with statevector method as the backend
backend = AerSimulator(method='matrix_product_state')

no_Spins = 8

# Define the lattice
line_latticec = LineLattice(num_nodes=no_Spins, boundary_condition=BoundaryCondition.PERIODIC)

# Construct the Heisenberg Hamiltonian
interaction_coefficients = (1, 1, 1)  # J on x, y, and z axes
external_magntic_field = (0, 0, 0)  # no magnetic field

heisenberg_model = HeisenbergModel(line_latticec, interaction_coefficients, external_magntic_field)

# Generate spin Hamiltonian to map it on the qubits
spin_ham = heisenberg_model.second_q_op()

print(f'The system is defined, number of spins: {no_Spins} \nHeisenberg Hamiltonian: {heisenberg_model.interaction_matrix()} \nSpin Hamiltonian: {spin_ham}')

mapper = LogarithmicMapper()
qubit_op = mapper.map(spin_ham)

print(f'The number of qubits in the qubit operator is {qubit_op.num_qubits}')

# These steps to prepare system for VQE
# Initialize the Estimator primitive (backend is handled internally)
estimator = Estimator()

ansatz = EfficientSU2(qubit_op.num_qubits, reps=2)

# Transpile the bound ansatz to optimize it for AerSimulator

# Prepare optimizer for VQE
optimizer = SLSQP(maxiter=200)

# Measure the time taken to compute the minimum eigenvalue
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)

# Start the timer
start_time = time.time()

# Compute the minimum eigenvalue
vqe_result = vqe.compute_minimum_eigenvalue(qubit_op)

# End the timer
end_time = time.time()

# Calculate time taken
time_taken = end_time - start_time

# Output the results
total_energy = vqe_result.eigenvalue.real
energy_per_spin = total_energy / no_Spins
print(f'Total Energy: {total_energy}')
print(f'Energy per Spin: {energy_per_spin}')
print(f'Time taken to compute the ground state energy: {time_taken:.4f} seconds')


The system is defined, number of spins: 8 
Heisenberg Hamiltonian: [[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]] 
Spin Hamiltonian: Spin Operator
spin=1/2, number spins=8, number terms=24
  1 * ( X_0 X_1 )
+ 1 * ( Y_0 Y_1 )
+ 1 * ( Z_0 Z_1 )
+ 1 * ( X_1 X_2 )
+ 1 * ( Y_1 Y_2 )
+ 1 * ( Z_1 Z_2 )
+ 1 * ( X_2 X_3 )
+ 1 * ( Y_2 Y_3 )
+ 1 * ( Z_2 Z_3 )
+ 1 * ( X_3 X_4 )
+ 1 * ( Y_3 Y_4 )
+ 1 * ( Z_3 Z_4 )
+ 1 * ( X_4 X_5 )
+ 1 * ( Y_4 Y_5 )
+ 1 * ( Z_4 Z_5 )
+ 1 * ( X_5 X_6 )
+ 1 * ( Y_5 Y_6 )
+ 1 * ( Z_5 Z_6 )
+ 1 * ( X_6 X_7 )
+ 1 * ( Y_6 Y_7 )
+ 1 * ( Z_6 Z_7 )
+ 1 * (

C:\Users\msi\AppData\Local\Temp\ipykernel_22668\3270293508.py:39: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


Total Energy: -3.2251473570144062
Energy per Spin: -0.4031434196268008
Time taken to compute the ground state energy: 18.1008 seconds


## This test to gauge the resources 

In [ ]:
import time
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_nature.second_q.hamiltonians import HeisenbergModel
from qiskit_nature.second_q.mappers import LogarithmicMapper
from qiskit_nature.second_q.hamiltonians.lattices import BoundaryCondition, LineLattice
from qiskit_ibm_runtime.runtime_job import 
from qiskit.primitives import Estimator

# Initialize AerSimulator for resource gauging
simulator = AerSimulator(method='statevector')

# Define VQE Hamiltonian and Mapper
no_Spins = 8
line_latticec = LineLattice(num_nodes=no_Spins, boundary_condition=BoundaryCondition.PERIODIC)
interaction_coefficients = (1, 1, 1)
external_magntic_field = (0, 0, 0)

heisenberg_model = HeisenbergModel(line_latticec, interaction_coefficients, external_magntic_field)
spin_ham = heisenberg_model.second_q_op()
mapper = LogarithmicMapper()
qubit_op = mapper.map(spin_ham)

# Ansatz and Transpile
ansatz = EfficientSU2(qubit_op.num_qubits, reps=2)
transpiled_ansatz = transpile(ansatz, backend=simulator, optimization_level=3)

# Output Resource Data
print(f"Circuit Depth: {transpiled_ansatz.depth()}")
print(f"Gate Count: {transpiled_ansatz.count_ops()}")

# Execute on Simulator
job = simulator.run(transpiled_ansatz)
job_monitor(job)
result = job.result()

# Retrieve Time Data
execution_time = result.time_taken
print(f"Execution Time on Simulator: {execution_time:.4f} seconds")


# Code with For-Loop

In [13]:
# Use AerSimulator with statevector method as the backend
backend = AerSimulator(method='matrix_product_state')

# Interaction coefficients and external magnetic field
interaction_coefficients = (1, 1, 1)  # J on x, y, and z axes
external_magntic_field = (0, 0, 0)  # No magnetic field

# Define mapper, optimizer, and estimator
mapper = LogarithmicMapper()
estimator = Estimator(options={'backend':backend})
optimizer = SLSQP(maxiter=200)

# Spin configurations to test
spins = [2,10]

# Results table
results = []

# Loop over spin numbers
for spin in spins:
    # Define the lattice and construct the Hamiltonian
    line_latticec = LineLattice(num_nodes=spin, boundary_condition=BoundaryCondition.PERIODIC)
    heisenberg_model = HeisenbergModel(line_latticec, interaction_coefficients, external_magntic_field)
    spin_ham = heisenberg_model.second_q_op()
    
    # Map the Hamiltonian to qubits
    qubit_op = mapper.map(spin_ham)
    
    # Define ansatz
    ansatz = EfficientSU2(qubit_op.num_qubits, reps=2)
    
    # Define VQE
    vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)
    
    # Start timing
    start_time = time.time()
    
    # Compute the minimum eigenvalue
    vqe_result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    # End timing
    end_time = time.time()
    
    # Calculate results
    time_taken = end_time - start_time
    total_energy = vqe_result.eigenvalue.real
    energy_per_spin = total_energy / spin
    
    # Store results
    results.append({
        "Spin Number": spin,
        "Total Ground State Energy": total_energy,
        "Ground State Energy per Spin": energy_per_spin,
        "Time Taken (s)": time_taken
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display the table
tools.display_dataframe_to_user(name="Ground State Energy Results", dataframe=results_df)


C:\Users\msi\AppData\Local\Temp\ipykernel_5780\2710644262.py:10: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={'backend':backend})


KeyboardInterrupt: 

# Tests

In [11]:
# Authenticate and initialize the Qiskit runtime service
service = QiskitRuntimeService()

# Define the number of spins in the lattice
no_spins = 2  # Number of spins in the 1D lattice

# Select a real backend with sufficient qubits
backend = service.least_busy(min_num_qubits=no_spins)
print(f"Using backend: {backend.name}")

Using backend: ibm_brisbane


In [12]:


# Define the lattice with periodic boundary conditions
line_lattice = LineLattice(num_nodes=no_spins, boundary_condition=BoundaryCondition.PERIODIC)

# Construct the Heisenberg Hamiltonian
interaction_coefficients = (1, 1, 1)  # J on x, y, and z axes
external_magnetic_field = (0, 0, 0)  # No external magnetic field
heisenberg_model = HeisenbergModel(line_lattice, interaction_coefficients, external_magnetic_field)

# Generate spin Hamiltonian and map to qubits
spin_ham = heisenberg_model.second_q_op()
mapper = LogarithmicMapper()
qubit_op = mapper.map(spin_ham)

# Print system information
print(f'The system is defined, number of spins: {no_spins}')
print(f'Heisenberg Hamiltonian Interaction Matrix:\n{heisenberg_model.interaction_matrix()}')
print(f'Spin Hamiltonian: {spin_ham}')
print(f'The number of qubits in the qubit operator: {qubit_op.num_qubits}')

# Define the ansatz (variational form) and optimizer
ansatz = EfficientSU2(qubit_op.num_qubits, reps=2)
optimizer = SLSQP(maxiter=200)

# Transpile the ansatz for the real backend
transpiled_ansatz = transpile(ansatz, backend=backend, optimization_level=3)
estimator = Estimator(options={"backend": backend, "shots": 1024})
# Initialize the VQE instance
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)

# Measure the time taken to compute the ground state energy
start_time = time.time()

# Submit VQE job to the real backend
print("Submitting VQE job to the real quantum computer...")
vqe_result = vqe.compute_minimum_eigenvalue(qubit_op)

# End the timer
end_time = time.time()

# Calculate and display results
total_energy = vqe_result.eigenvalue.real
energy_per_spin = total_energy / no_spins
time_taken = end_time - start_time

print(f'\n=== Results ===')
print(f'Total Energy: {total_energy:.6f}')
print(f'Energy per Spin: {energy_per_spin:.6f}')
print(f'Time taken to compute the ground state energy: {time_taken:.4f} seconds')


The system is defined, number of spins: 2
Heisenberg Hamiltonian Interaction Matrix:
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
Spin Hamiltonian: Spin Operator
spin=1/2, number spins=2, number terms=3
  1 * ( X_0 X_1 )
+ 1 * ( Y_0 Y_1 )
+ 1 * ( Z_0 Z_1 )
The number of qubits in the qubit operator: 2


RequestsApiError: 'HTTPSConnectionPool(host=\'api.quantum.ibm.com\', port=443): Max retries exceeded with url: /runtime/backends/ibm_brisbane/properties (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000015A4B64A210>: Failed to resolve \'api.quantum.ibm.com\' ([Errno 11001] getaddrinfo failed)"))'

In [1]:
pip install pandoc

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pandoc: filename=pandoc-2.4-py3-none-any.whl size=34819 sha256=1c860fb9e777c5774e79265d2c80497ef11c3d66e8cd0245080f1e2c02d3eede
  Stored in directory: c:\users\msi\appdata\local\pip\cache\wheels\9c\2f\9f\b1aac8c3e74b4ee327dc8c6eac5128996f9eadf586e2c0ba67
Successfully built pandoc
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
